In [1]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"Implicit", category=UserWarning)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from ipynb.fs.full.dynamic_recourse import DynamicCsvCatalog, DynamicMLModelCatalog, RecourseExperiment

# os.environ["CUDA_VISIBLE_DEVICES"] = ""

#     torch.manual_seed(0)
#     random.seed(0)
#     np.random.seed(0)

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


Using TensorFlow backend.


In [3]:
# Load the dataset from a csv file
dataset = DynamicCsvCatalog(file_path="../datasets/unimodal_dataset_5.csv", 
                            continuous=['feature1', 'feature2'], categorical=[],
                            immutables=[], target='target', test_size=0.3)

In [ ]:
hyper_params = {'learning_rate': 0.04, 'epochs': 4, 'batch_size': 1, 'hidden_size': [4]}
dice_params = {"num": 1}
wachter_params = {"loss_type": "BCE", "t_max_min": 1.0}

kwargs = {
    'hyper_params': hyper_params,
    'dice_params': dice_params,
    'wachter_params': wachter_params
}

experiment = RecourseExperiment(dataset, experiment_name='p_value', **kwargs)
experiment.run(total_recourse=0.85, recourse_per_epoch=0.05)
experiment.save_data()

[INFO] Training the initial model [dynamic_recourse.ipynb train_model]
balance on test set 0.5142857142857142, balance on test set 0.4666666666666667
Epoch 0/3
----------
train Loss: 0.1166 Acc: 0.9643

test Loss: 0.0132 Acc: 1.0000

Epoch 1/3
----------
train Loss: 0.0061 Acc: 1.0000

test Loss: 0.0027 Acc: 1.0000

Epoch 2/3
----------
train Loss: 0.0011 Acc: 1.0000

test Loss: 0.0013 Acc: 1.0000

Epoch 3/3
----------
train Loss: 0.0014 Acc: 1.0000

test Loss: 0.0005 Acc: 1.0000

benchmark 0.0 -- DICE 0.0


C:\Users\Olek\anaconda3\envs\rp\lib\site-packages\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  f"cannot re-order features for non dataframe input: {type(x)}"


[INFO] Starting epoch 1 [dynamic_recourse.ipynb run]
[INFO] Applying the DICE generator. [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 1 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 2 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 3 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 4 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 5 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 6 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Updating the DICE model [dynamic_recourse.ipynb update_model]
Loaded model from C:\Users\Olek\carla\models\custom\ann_layers_4.pt
test accuracy for model: 1.0
[INFO] Current balance: train set 0.5357142857142857, test set 0.4666666666666667 [dynamic_recourse.ipyn

C:\Users\Olek\anaconda3\envs\rp\lib\site-packages\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  f"cannot re-order features for non dataframe input: {type(x)}"


[INFO] Starting epoch 2 [dynamic_recourse.ipynb run]
[INFO] Applying the DICE generator. [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 1 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 2 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 3 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 4 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 5 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 6 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Updating the DICE model [dynamic_recourse.ipynb update_model]
Loaded model from C:\Users\Olek\carla\models\custom\ann_layers_4.pt
test accuracy for model: 1.0
[INFO] Current balance: train set 0.5571428571428572, test set 0.4666666666666667 [dynamic_recourse.ipyn

C:\Users\Olek\anaconda3\envs\rp\lib\site-packages\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  f"cannot re-order features for non dataframe input: {type(x)}"


[INFO] Starting epoch 3 [dynamic_recourse.ipynb run]
[INFO] Applying the DICE generator. [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 1 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 2 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 3 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 4 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 5 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 6 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Updating the DICE model [dynamic_recourse.ipynb update_model]
Loaded model from C:\Users\Olek\carla\models\custom\ann_layers_4.pt
test accuracy for model: 1.0
[INFO] Current balance: train set 0.5785714285714286, test set 0.4666666666666667 [dynamic_recourse.ipyn

C:\Users\Olek\anaconda3\envs\rp\lib\site-packages\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  f"cannot re-order features for non dataframe input: {type(x)}"


[INFO] Starting epoch 4 [dynamic_recourse.ipynb run]
[INFO] Applying the DICE generator. [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 1 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 2 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 3 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 4 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 5 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Generating counterfactual 6 with DICE [dynamic_recourse.ipynb apply_recourse_with_timeout]
[INFO] Updating the DICE model [dynamic_recourse.ipynb update_model]
Loaded model from C:\Users\Olek\carla\models\custom\ann_layers_4.pt
test accuracy for model: 1.0
[INFO] Current balance: train set 0.6, test set 0.4666666666666667 [dynamic_recourse.ipynb retrain]
Epoc